# CAD PROJECT- Classical Approach
*By Jaime Simarro & Ahmed Gouda*

With this script you can find:
* Try diferences normalizations approach
* Try diferences models (SVM and Random Forest)
* Combination of desired features and color spaces 
* Save the results in the excel file


In [0]:
from google.colab import drive

drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [0]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
import math
!pip install ciecam02
from ciecam02 import rgb2jch, jch2rgb
from skimage.feature import greycomatrix, greycoprops
! pip install xlutils


In [0]:
#### JUST CHANGE THIS #####

dermo_challenge=False
number_Normalization_type=1
which_feat_calculate=[5]
#which_feat_calculate=[0,1,2,3,4]
which_color_calculate=[4,5]
#which_color_calculate=[5]

###########################
switcher_feat = {
    0: "GLCM",
    1: "Histogram",
    2: "Fintesity",
    3: "Fstat",
    4: "Gabour",
    5: "Combination of features",
}

if (dermo_challenge):
  switcher_color_space = {
      0: "Frgb",
      1: "Fhsv",
      2: "Flab",
      3: "Fjch",  
      4: "Fhybrid",        
      5: "Fsix",
      6: "Combination of color spaces", 
  }
  normalization_type=''
else:
  switcher_color_space = {
      0: "Frgb",
      1: "Fhsv",
      2: "Flab",
      3: "Fjch",
      4: "Ftissue",    
      5: "Combination of color spaces", 

  }
  switcher_normalization = {
    0: "Original",
    1: "Reinhard",
    2: "Macenko",
  }
  normalization_type=switcher_normalization.get(number_Normalization_type, "Invalid normalization type")

In [0]:
if (dermo_challenge):
  path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/train/nv/*.jpg'  # Nevus
  path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Challenge/train/les/*.jpg' # Others
  path_general_feat='/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo Features/'
else:
  if normalization_type==1: #Original
    path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/train/m0/*.png' # Malignan
    path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/train/b0/*.png' # Belignan
  if normalization_type==2: #Reinhard
    path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/train/m0/*.png' # Malignan
    path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Reinhard/train/b0/*.png' # Belignan
  if normalization_type==3: #Macenko
    path_data_pos='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Macenko/train/m0/*.png' # Malignan
    path_data_neg='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge Macenko/train/b0/*.png' # Belignan
  path_general_feat='/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Features/'
path4load_general=path_general_feat+normalization_type+'/One/train/'


In [0]:
def load_combine_feat(folder_class,combine_number_Feat,combine_number_Color):
  if isinstance(combine_number_Feat,int)  : # For int
    combine_number_Feat=[combine_number_Feat]
  if isinstance(combine_number_Color,int)  : # For int
    combine_number_Color=[combine_number_Color]
  print(combine_number_Feat)
  for c in range(len(combine_number_Color)):
    for f in range(len(combine_number_Feat)):
      feat_type=switcher_feat.get(combine_number_Feat[f], "Invalid feature type")
      color_space_type=switcher_color_space.get(combine_number_Color[c], "Invalid color space type")
      path4load=path4load_general+ folder_class+color_space_type+'_'+feat_type+'.npy'
      if(f==0 and c==0):
        X = np.load(path4load, allow_pickle=True)
      else:
        X=np.concatenate((X,np.load(path4load, allow_pickle=True)),axis=1)
  return X

def load_one_feat(folder_class,fnumber_Feat,fnumber_Color):
  feat_type=switcher_feat.get(fnumber_Feat, "Invalid feature type")
  color_space=switcher_color_space.get(fnumber_Color, "Invalid color space type")
  path4load=path4load_general+folder_class +color_space+'_'+feat_type+'.npy'
  X = np.load(path4load, allow_pickle=True)
  return X



In [0]:
def compute_acc(fnumber_Feat_type,fnumber_Color_type,combine):
  if combine:
    X_pos=load_combine_feat('pos/',fnumber_Feat_type,fnumber_Color_type)
    X_neg=load_combine_feat('neg/',fnumber_Feat_type,fnumber_Color_type) 
  else:
    X_pos=load_one_feat('pos/',fnumber_Feat_type,fnumber_Color_type)
    X_neg=load_one_feat('neg/',fnumber_Feat_type,fnumber_Color_type)
  # print("Positive matrix dimension: ", X_pos.shape)
  # print("Negative matrix dimension: ", X_neg.shape)
  ## Concatenate and Shuffed
  rnd = np.random.RandomState(357)
  X=np.concatenate((X_pos,X_neg))
  y_pos=np.ones(X_pos.shape[0])
  y_neg=np.zeros(X_neg.shape[0])
  y=np.concatenate((y_pos,y_neg))
  shuffled_indices = rnd.permutation(np.arange(y.shape[0]))
  X=X[shuffled_indices]
  y=y[shuffled_indices]
  ## Normalization
  if (True):
    mean_X  = np.mean(X, 0)
    std_X   = np.std(X,0)
    X = (X-mean_X)/std_X
  else:
    from sklearn.preprocessing import Normalizer
    transformer = Normalizer(norm='l2').fit(X)  # fit does nothing
    X=transformer.transform(X)
  ## CV
  from sklearn import svm

  from sklearn.ensemble import RandomForestClassifier
  from sklearn.model_selection import cross_validate
  if(True):
    clf = RandomForestClassifier(n_estimators=500, max_depth=50,random_state=0)
    cv_results = cross_validate(clf, X, y, scoring = ['accuracy', 'precision','recall'],cv=5,return_train_score=True)
  else:
    clf = svm.SVC(gamma='scale', random_state=0)
    cv_results = cross_validate(clf, X, y, scoring = ['accuracy', 'precision','recall'],cv=5,return_train_score=True)

  test_accuracy=np.mean(cv_results['test_accuracy'])*100
  test_accuracy_std=np.std(cv_results['test_accuracy'])*2*100

  s=str(round(test_accuracy,2)) + ' +/- ' + str(round(test_accuracy_std,2))
  print('Test',fnumber_Feat_type,fnumber_Color_type)
  print('Accuracy ',s)
  return s

# Combination of desired features and color spaces & saving the results in the excel file

In [0]:
from xlutils.copy import copy # http://pypi.python.org/pypi/xlutils
from xlwt import Workbook 
from xlrd import open_workbook # http://pypi.python.org/pypi/xlrd

excel_path=path_general_feat+' Results_RF500_50_'+normalization_type+'.xlsx'
if os.path.isfile(excel_path):
  # If Workbook exists, it is updated
  rb = open_workbook(excel_path)
  r_sheet = rb.sheet_by_index(0) # read only copy to introspect the file
  wb = copy(rb) # a writable copy (I can't read values out of this, only write to it)
  w_sheet = wb.get_sheet(0) # the sheet to write to within the writable copy 
else:
  # Otherwise, workbook is created 
  wb = Workbook() 
  # add_sheet is used to create sheet. 
  w_sheet = wb.add_sheet('Sheet 1') 

for f_s in range(len(switcher_feat)):
  w_sheet.write(f_s+1,0 ,switcher_feat.get(f_s))
for c_s in range(len(switcher_color_space)):
  w_sheet.write(0,c_s+1 ,switcher_color_space.get(c_s))

for f in range(len(which_feat_calculate)):
  for c in range(len(which_color_calculate)): 
    combine=False 
    if(which_feat_calculate[f]+1)==len(switcher_feat):
      list_feat=range(which_feat_calculate[f])
      combine=True
    else:
      list_feat=which_feat_calculate[f]
    if ((which_color_calculate[c]+1)==len(switcher_color_space)):
      list_color=range(which_color_calculate[c])
      combine=True
    else:
      list_color=which_color_calculate[c]
    s=compute_acc(list_feat,list_color,combine)
    w_sheet.write(which_feat_calculate[f]+1,which_color_calculate[c]+1,s)

wb.save(excel_path) 

range(0, 5)
range(0, 5)
Test range(0, 5) 4
Accuracy  87.7 +/- 1.11
range(0, 5)
range(0, 5)
Test range(0, 5) range(0, 5)
Accuracy  89.28 +/- 1.25
